# Analisis Algoritma Knapsack 0/1 (Lab Inventory)


# 1. Setup & Import Library


In [1]:
import json
import time
import random
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from typing import List, Tuple, Dict, Any

# 2. Load Instance dari Folder `data/`


In [2]:
def load_instance(file_path: str) -> Dict[str, Any]:
    with open(file_path, 'r', encoding='utf=8') as f:
        return json.load(f)

In [3]:
instance_path = "../DAA_Kelompok6_KelasB/data/knapsack_labA_inv.json"
instance = load_instance(instance_path) 
print("Instance loaded:")
print(json.dumps(instance, indent=2))

Instance loaded:
{
  "project": "knapsack_01_labA",
  "capacity": 50,
  "description": "Lab Inventory A - 30 items with varying value/weight ratios",
  "items": [
    {
      "id": "LAB-A-001",
      "value": 25,
      "weight": 8
    },
    {
      "id": "LAB-A-002",
      "value": 18,
      "weight": 5
    },
    {
      "id": "LAB-A-003",
      "value": 30,
      "weight": 12
    },
    {
      "id": "LAB-A-004",
      "value": 22,
      "weight": 7
    },
    {
      "id": "LAB-A-005",
      "value": 35,
      "weight": 15
    },
    {
      "id": "LAB-A-006",
      "value": 16,
      "weight": 4
    },
    {
      "id": "LAB-A-007",
      "value": 28,
      "weight": 10
    },
    {
      "id": "LAB-A-008",
      "value": 20,
      "weight": 6
    },
    {
      "id": "LAB-A-009",
      "value": 32,
      "weight": 13
    },
    {
      "id": "LAB-A-010",
      "value": 24,
      "weight": 9
    },
    {
      "id": "LAB-A-011",
      "value": 15,
      "weight": 3
    },
    {
  

# 3. Implementasi Algoritma Knapsack


In [4]:
# ==================== GREEDY (Density) ====================

def knapsack_greedy(items: List[Dict], capacity: int) -> Tuple[List[int], int]:
    """
    Greedy berdasarkan density = value / weight.
    """
    sorted_items = sorted(items, key=lambda x: x['value'] / x['weight'], reverse=True)
    total_weight = 0
    total_value = 0
    selected = [0] * len(items)
    
    for idx, item in enumerate(sorted_items):
        if total_weight + item['weight'] <= capacity:
            selected[items.index(item)] = 1
            total_weight += item['weight']
            total_value += item['value']
    
    return selected, total_value

In [5]:
# ==================== DYNAMIC PROGRAMMING ====================
def knapsack_dp(items: List[Dict], capacity: int) -> Tuple[List[int], int]:
    n = len(items)
    dp = [[0] * (capacity + 1) for _ in range(n + 1)]
    
    for i in range(1, n + 1):
        w = items[i-1]['weight']
        v = items[i-1]['value']
        for c in range(capacity + 1):
            if w > c:
                dp[i][c] = dp[i-1][c]
            else:
                dp[i][c] = max(dp[i-1][c], dp[i-1][c - w] + v)
    
    # Backtrack
    selected = [0] * n
    c = capacity
    for i in range(n, 0, -1):
        if dp[i][c] != dp[i-1][c]:
            selected[i-1] = 1
            c -= items[i-1]['weight']
    
    return selected, dp[n][capacity]

In [6]:
# ==================== BRANCH & BOUND ====================
def knapsack_branch_bound(items: List[Dict], capacity: int) -> Tuple[List[int], int]:
    """
    Branch & Bound dengan bound berdasarkan fractional knapsack.
    """
    n = len(items)
    items_sorted = sorted(items, key=lambda x: x['value'] / x['weight'], reverse=True)
    
    best_value = 0
    best_selection = [0] * n
    
    def backtrack(i, current_weight, current_value, selection):
        nonlocal best_value, best_selection
        
        if i == n:
            if current_value > best_value:
                best_value = current_value
                best_selection = selection.copy()
            return
        
        # Bound: fractional knapsack upper bound
        bound = current_value
        remaining = capacity - current_weight
        j = i
        while j < n and remaining >= items_sorted[j]['weight']:
            bound += items_sorted[j]['value']
            remaining -= items_sorted[j]['weight']
            j += 1
        if j < n:
            bound += remaining * (items_sorted[j]['value'] / items_sorted[j]['weight'])
        
        if bound <= best_value:
            return
        
        # Pilih item i
        if current_weight + items_sorted[i]['weight'] <= capacity:
            selection.append(i)
            backtrack(i + 1, 
                      current_weight + items_sorted[i]['weight'], 
                      current_value + items_sorted[i]['value'], 
                      selection)
            selection.pop()
        
        # Lewati item i
        backtrack(i + 1, current_weight, current_value, selection)
    
    backtrack(0, 0, 0, [])
    
    # Map kembali ke index asli
    final_selection = [0] * n
    for idx in best_selection:
        original_idx = items.index(items_sorted[idx])
        final_selection[original_idx] = 1
    
    return final_selection, best_value


# 4. Evaluasi & Perbandingan Waktu Eksekusi


In [ ]:
def evaluate_algorithms(instance: Dict[str, Any]):
    items = instance['items']
    capacity = instance['capacity']
    results = {}
    
    algorithms = {
        'Greedy': knapsack_greedy,
        'DP': knapsack_dp,
        'Branch&Bound': knapsack_branch_bound
    }
    
    for name, algo in algorithms.items():
        start = time.perf_counter()
        selection, value = algo(items, capacity)
        elapsed = (time.perf_counter() - start) * 1000  # ms
        
        results[name] = {
            'value': value,
            'time_ms': elapsed,
            'selection': selection
        }
        
    return results